In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/earthquake.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442282 entries, 0 to 442281
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   time             442282 non-null  object 
 1   latitude         442282 non-null  float64
 2   longitude        442282 non-null  float64
 3   depth            442277 non-null  float64
 4   mag              442282 non-null  float64
 5   magType          442176 non-null  object 
 6   nst              190860 non-null  float64
 7   gap              140964 non-null  float64
 8   dmin             59855 non-null   float64
 9   rms              331554 non-null  float64
 10  net              442282 non-null  object 
 11  id               442281 non-null  object 
 12  updated          442281 non-null  object 
 13  place            0 non-null       float64
 14  type             0 non-null       float64
 15  horizontalError  0 non-null       float64
 16  depthError       0 non-null       floa

In [ ]:
df = df.drop(columns=['id', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'magNst',
                      'status', 'locationSource', 'magSource'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442282 entries, 0 to 442281
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   time       442282 non-null  object 
 1   latitude   442282 non-null  float64
 2   longitude  442282 non-null  float64
 3   depth      442277 non-null  float64
 4   mag        442282 non-null  float64
 5   magType    442176 non-null  object 
 6   nst        190860 non-null  float64
 7   gap        140964 non-null  float64
 8   dmin       59855 non-null   float64
 9   rms        331554 non-null  float64
 10  net        442282 non-null  object 
dtypes: float64(8), object(3)
memory usage: 37.1+ MB


In [ ]:
df['mag'].mean()

np.float64(3.925040856286261)

In [ ]:
def convert_to_mw(row):
    if row['magType'] == 'ml':
        return 0.85 * row['mag'] + 0.15
    elif row['magType'] == 'mb':
        return 0.85 * row['mag'] + 0.33
    else:
        # fallback: if magType is missing or unknown, keep original mag
        return row['mag']

# Apply the conversion to create a new column
df['mag_in_mw'] = df.apply(convert_to_mw, axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442282 entries, 0 to 442281
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   time       442282 non-null  object 
 1   latitude   442282 non-null  float64
 2   longitude  442282 non-null  float64
 3   depth      442277 non-null  float64
 4   mag        442282 non-null  float64
 5   magType    442176 non-null  object 
 6   nst        190860 non-null  float64
 7   gap        140964 non-null  float64
 8   dmin       59855 non-null   float64
 9   rms        331554 non-null  float64
 10  net        442282 non-null  object 
 11  mag_in_mw  442282 non-null  float64
dtypes: float64(9), object(3)
memory usage: 40.5+ MB


In [ ]:
df['time'] = pd.to_datetime(df['time'])


In [ ]:
df = df.sort_values('time').reset_index(drop=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442282 entries, 0 to 442281
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   time       442282 non-null  datetime64[ns, UTC]
 1   latitude   442282 non-null  float64            
 2   longitude  442282 non-null  float64            
 3   depth      442277 non-null  float64            
 4   mag        442282 non-null  float64            
 5   magType    442176 non-null  object             
 6   nst        190860 non-null  float64            
 7   gap        140964 non-null  float64            
 8   dmin       59855 non-null   float64            
 9   rms        331554 non-null  float64            
 10  net        442282 non-null  object             
 11  mag_in_mw  442282 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(9), object(2)
memory usage: 40.5+ MB


# Seismic labeling

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.spatial import cKDTree

# -----------------------------
# 1️⃣ Parameters (tune as needed)
# -----------------------------
TIME_WINDOW_DAYS = 30        # max time difference to consider sequences
DISTANCE_THRESHOLD_KM = 50   # max distance to consider same sequence

# -----------------------------
# 2️⃣ Initialize new columns
# -----------------------------
df['is_mainshock'] = 0
df['is_foreshock'] = 0
df['is_aftershock'] = 0

# Convert time to datetime if not already
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

# -----------------------------
# 3️⃣ Precompute radians for all lat/lon
# -----------------------------
lat_rad = np.radians(df['latitude'].values)
lon_rad = np.radians(df['longitude'].values)

# For Haversine, we convert distance threshold to radians
EARTH_RADIUS_KM = 6371.0
DIST_THRESHOLD_RAD = DISTANCE_THRESHOLD_KM / EARTH_RADIUS_KM

# -----------------------------
# 4️⃣ Build KDTree for fast spatial queries
# -----------------------------
# Use 3D Cartesian coordinates on unit sphere for KDTree
x = np.cos(lat_rad) * np.cos(lon_rad)
y = np.cos(lat_rad) * np.sin(lon_rad)
z = np.sin(lat_rad)
coords = np.column_stack((x, y, z))
tree = cKDTree(coords)

# -----------------------------
# 5️⃣ Vectorized labeling loop
# -----------------------------
print("Starting optimized seismic labeling...")

mag = df['mag_in_mw'].values
time = df['time'].values.astype('datetime64[D]')  # approximate day resolution

n = len(df)

for i in tqdm(range(n), desc="Processing events"):
    # Current event
    t0 = time[i]
    m0 = mag[i]

    # Find future events within TIME_WINDOW_DAYS
    mask_time = (time > t0) & (time <= t0 + np.timedelta64(TIME_WINDOW_DAYS, 'D'))
    if not np.any(mask_time):
        df.at[i, 'is_mainshock'] = 1
        continue

    future_idx = np.where(mask_time)[0]

    # Query spatially within distance threshold
    idxs_within_dist = tree.query_ball_point(coords[i], DIST_THRESHOLD_RAD)

    # Only consider future events that are also nearby
    nearby_future_idx = np.intersect1d(future_idx, idxs_within_dist)

    if len(nearby_future_idx) == 0:
        df.at[i, 'is_mainshock'] = 1
        continue

    # Check magnitudes
    future_mags = mag[nearby_future_idx]
    max_future_mag = future_mags.max()

    if m0 >= max_future_mag:
        # Current event is mainshock
        df.at[i, 'is_mainshock'] = 1
    else:
        # Current event is foreshock
        df.at[i, 'is_foreshock'] = 1
        mainshock_idx = nearby_future_idx[future_mags.argmax()]
        df.at[mainshock_idx, 'is_mainshock'] = 1

        # Mark aftershocks
        # Future events within distance & TIME_WINDOW_DAYS after mainshock
        mask_after = (time > time[mainshock_idx]) & \
                     (time <= time[mainshock_idx] + np.timedelta64(TIME_WINDOW_DAYS, 'D'))
        after_idx = np.intersect1d(np.where(mask_after)[0], tree.query_ball_point(coords[mainshock_idx], DIST_THRESHOLD_RAD))
        df.loc[after_idx, 'is_aftershock'] = 1

print("Optimized seismic labeling complete!")


Starting optimized seismic labeling...


Processing events: 100%|██████████| 442282/442282 [29:23<00:00, 250.74it/s]

Optimized seismic labeling complete!


In [ ]:
df.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'mag_in_mw', 'is_mainshock',
       'is_foreshock', 'is_aftershock'],
      dtype='object')

In [ ]:
df['is_mainshock'].value_counts(normalize=True)

,proportion
is_mainshock,
1,0.582922
0,0.417078


In [ ]:
df['is_foreshock'].value_counts(normalize=True)

,proportion
is_foreshock,
0,0.547703
1,0.452297


In [ ]:
df['is_aftershock'].value_counts(normalize=True)

,proportion
is_aftershock,
0,0.635477
1,0.364523


In [ ]:
df.to_csv('earthquake_1930to2018_seismicLabeled.csv',index=False)

In [ ]:
df = pd.read_csv('/content/earthquake_1930to2018_seismicLabeled.csv')

In [ ]:
df['time'] = pd.to_datetime(df['time'], format='ISO8601')
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['dayofweek'] = df['time'].dt.dayofweek  # 0 = Monday, 6 = Sunday
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df['second'] = df['time'].dt.second

# -----------------------------
# 2️⃣ Time since previous earthquake (in seconds)
# -----------------------------
df = df.sort_values('time').reset_index(drop=True)
df['time_since_prev_quake'] = df['time'].diff().dt.total_seconds()
df['time_since_prev_quake'].fillna(0, inplace=True)  # first quake has no previous

# -----------------------------
# 3️⃣ Optional: elapsed days instead of seconds
# -----------------------------
df['days_since_prev_quake'] = df['time_since_prev_quake'] / (24*3600)

/tmp/ipython-input-2868485514.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['time_since_prev_quake'].fillna(0, inplace=True)  # first quake has no previous


# Imputation

In [ ]:
df.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'mag_in_mw', 'is_mainshock',
       'is_foreshock', 'is_aftershock', 'month', 'day', 'dayofweek', 'hour',
       'minute', 'second', 'time_since_prev_quake', 'days_since_prev_quake'],
      dtype='object')

In [ ]:
target = 'is_mainshock'
target_copy = df[target]
df = df.drop(columns=[target])

In [ ]:
new_df = df.drop(columns=['is_foreshock', 'is_aftershock', 'mag', 'magType', 'net', 'time'])
new_df.columns

Index(['latitude', 'longitude', 'depth', 'nst', 'gap', 'dmin', 'rms',
       'mag_in_mw', 'month', 'day', 'dayofweek', 'hour', 'minute', 'second',
       'time_since_prev_quake', 'days_since_prev_quake'],
      dtype='object')

In [ ]:
new_df[target] = target_copy

In [ ]:
new_df.columns

Index(['latitude', 'longitude', 'depth', 'nst', 'gap', 'dmin', 'rms',
       'mag_in_mw', 'month', 'day', 'dayofweek', 'hour', 'minute', 'second',
       'time_since_prev_quake', 'days_since_prev_quake', 'is_mainshock'],
      dtype='object')

In [ ]:
new_df.isna().sum()

,0
latitude,0
longitude,0
depth,5
nst,251422
gap,301318
dmin,382427
rms,110728
mag_in_mw,0
month,0
day,0


In [ ]:
new_df.dtypes

,0
latitude,float64
longitude,float64
depth,float64
nst,float64
gap,float64
dmin,float64
rms,float64
mag_in_mw,float64
month,int32
day,int32


In [ ]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from tqdm import tqdm

# -----------------------------
# 1️⃣ Prepare features to impute
# -----------------------------
X = new_df.drop(columns=['is_mainshock']).values  # use NumPy array for speed
n_rows = X.shape[0]

# -----------------------------
# 2️⃣ Initialize Iterative Imputer
# -----------------------------
imputer = IterativeImputer(max_iter=10, random_state=42, sample_posterior=False)

# -----------------------------
# 3️⃣ Chunked imputation with progress bar
# -----------------------------
chunk_size = 50000
imputed_chunks = []

print("Starting iterative imputation with progress bar...")
for start in tqdm(range(0, n_rows, chunk_size), desc="Imputing chunks"):
    end = min(start + chunk_size, n_rows)
    X_chunk = X[start:end]
    imputed_chunk = imputer.fit_transform(X_chunk)
    imputed_chunks.append(imputed_chunk)

# -----------------------------
# 4️⃣ Combine all chunks
# -----------------------------
X_imputed = np.vstack(imputed_chunks)

# -----------------------------
# 5️⃣ Replace in original DataFrame
# -----------------------------
new_df.iloc[:, 0:-1] = X_imputed  # all columns except 'is_mainshock'

print("Iterative imputation complete!")


Starting iterative imputation with progress bar...


Imputing chunks: 100%|██████████| 9/9 [01:01<00:00,  6.79s/it]

Iterative imputation complete!


In [ ]:
new_df.isna().sum()

,0
latitude,0
longitude,0
depth,0
nst,0
gap,0
dmin,0
rms,0
mag_in_mw,0
month,0
day,0


# Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# 1️⃣ Separate features and target
# -----------------------------
X = new_df.drop(columns=['is_mainshock']).values
y = new_df['is_mainshock'].values

# -----------------------------
# 2️⃣ Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -----------------------------
# 3️⃣ Feature scaling
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -----------------------------
# 4️⃣ Train Logistic Regression
# -----------------------------
clf = LogisticRegression(
    solver='saga',          # efficient for large datasets
    max_iter=1000,
    class_weight='balanced', # handle class imbalance
    n_jobs=-1
)
clf.fit(X_train_scaled, y_train)

# -----------------------------
# 5️⃣ Predictions and evaluation
# -----------------------------
y_pred = clf.predict(X_test_scaled)

print("Classification Report for Mainshock Prediction:\n")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Classification Report for Mainshock Prediction:

              precision    recall  f1-score   support

           0       0.60      0.67      0.63     36893
           1       0.74      0.68      0.71     51564

    accuracy                           0.68     88457
   macro avg       0.67      0.67      0.67     88457
weighted avg       0.68      0.68      0.68     88457

Confusion Matrix:

[[24700 12193]
 [16502 35062]]


In [ ]:
new_df.head()

,latitude,longitude,depth,nst,gap,dmin,rms,mag_in_mw,month,day,dayofweek,hour,minute,second,time_since_prev_quake,days_since_prev_quake,is_mainshock
0,-4.610,153.176,35.0,-608.559724,2877.084524,19.646499,13.237037,6.5,1,18,5,7,4,2,0.0,0.000000,1
1,51.389,179.824,25.0,-715.562304,3334.704429,21.525369,14.764444,6.4,2,2,6,14,56,2,1324320.0,15.327778,1
2,-21.871,-175.099,35.0,173.404604,-41.541991,-0.951327,-1.845603,6.4,2,14,4,20,41,15,1057513.0,12.239734,1
3,-33.292,-178.005,15.0,182.403831,0.331234,-0.602494,-1.790864,6.3,3,6,3,15,35,7,1709632.0,19.787407,1
4,-7.738,125.808,10.0,-553.268056,2863.375434,19.116453,12.437991,7.0,3,26,2,7,12,8,1697821.0,19.650706,1


In [ ]:
new_df.to_csv('mainshock_labeled_clean_data.csv',index=False)

# Group members

File Serial 01: XGBoost and LGBoost (Afia): https://colab.research.google.com/drive/1pkaNMvaYaPC4VTB55xDnFiti9axrzOFs?usp=sharing